In [22]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import davies_bouldin_score, silhouette_score

from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

from itertools import product
from energy_consumption_architecture.utils.paths import data_dir
# from kneebow.rotor import Rotor


## Funciones auxiliares

In [23]:
def optimal_k_selection(X, max_k=10):
    """
    Calcula el número óptimo de clusters usando el índice de silueta y el método del codo.

    Parámetros:
    - X: Dataset (matriz de características).
    - max_k: Número máximo de clusters a evaluar. Por defecto es 10.

    Retorna:
    - optimal_k: Número óptimo de clusters seleccionado.
    """

    Sum_of_squared_distances = []
    silhouette_scores = []
    K_range = range(2, max_k + 1)

    for k in K_range:
        km = KMeans(n_clusters=k, random_state=42)
        y = km.fit_predict(X)
        Sum_of_squared_distances.append(km.inertia_)
        silhouette_scores.append(silhouette_score(X, y))

    # Determinación del número óptimo de clusters según el índice de silueta
    optimal_k_silhouette = K_range[np.argmax(silhouette_scores)]

    # Determinación del número óptimo de clusters según el método del codo
    inertia_differences = np.diff(Sum_of_squared_distances)
    optimal_k_elbow = K_range[np.argmin(inertia_differences) + 1]  # +1 para ajustar el índice

    # Selección de un solo valor de K
    if optimal_k_silhouette == optimal_k_elbow:
        optimal_k = optimal_k_silhouette
    else:
        optimal_k = optimal_k_silhouette  # En caso de diferencia, priorizamos el índice de silueta

    return optimal_k


In [ ]:
def optimal_dbscan_params(features, eps_range=(0.05, 0.2, 0.05), min_samples_range=(3, 12)):
    """
    Encuentra los parámetros óptimos para DBSCAN (eps y min_samples) basados en el índice de silueta.

    Parámetros:
    - features: Matriz de características para clustering.
    - eps_range: Tupla con rango de valores de eps (inicio, fin, paso).
    - min_samples_range: Tupla con valores de min_samples (inicio, fin).

    Retorna:
    - Mejor combinación de (eps, min_samples) basada en el índice de silueta.
    """

    # Paso 1: Gráfica de distancia de vecinos para estimación inicial de `eps`
    neighbors = NearestNeighbors(n_neighbors=2)
    neighbors_fit = neighbors.fit(features)
    distances, _ = neighbors_fit.kneighbors(features)

    # Paso 2: Pruebas de diferentes combinaciones de eps y min_samples
    eps_values = np.arange(*eps_range)
    min_samples_values = np.arange(*min_samples_range)
    dbscan_params = list(product(eps_values, min_samples_values))
    
    best_params = (None, None)
    best_sil_score = -1  # Inicializamos con un valor muy bajo

    # Almacenamos métricas para análisis adicional
    results = {
        'Eps': [],
        'Min_samples': [],
        'Silhouette Score': [],
        'Clusters': []
    }

    for eps, min_samples in dbscan_params:
        db = DBSCAN(eps=eps, min_samples=min_samples)
        labels = db.fit_predict(features)

        # Solo evaluamos si hay más de un cluster
        if len(set(labels)) > 1:
            try:
                sil_score = silhouette_score(features, labels)
            except ValueError:
                sil_score = 0  # Si no se puede calcular, asignamos 0
        else:
            sil_score = 0

        # Guardamos resultados en el diccionario
        results['Eps'].append(eps)
        results['Min_samples'].append(min_samples)
        results['Silhouette Score'].append(sil_score)
        results['Clusters'].append(len(set(labels)))

        # Actualizar los mejores parámetros si encontramos un mejor índice de silueta
        if sil_score > best_sil_score:
            best_sil_score = sil_score
            best_params = (eps, min_samples)

    # Convertimos resultados en DataFrame para análisis
    df_results = pd.DataFrame(results)

    # Resultados de pivot para visualización opcional
    pivot_sil_score = pd.pivot_table(df_results, values='Silhouette Score', columns='Eps', index='Min_samples')
    pivot_clusters = pd.pivot_table(df_results, values='Clusters', columns='Eps', index='Min_samples')

    return best_params


In [24]:
def optimal_clusters_hierarchical(features, method='ward', last_n=10):
    """
    Calcula el número óptimo de clusters para clustering jerárquico usando la aceleración en la linkage matrix.

    Parámetros:
    - features: Matriz de características para clustering.
    - method: Método de linkage. Por defecto es 'ward'.
    - last_n: Número de fusiones a considerar para calcular el número óptimo de clusters. Por defecto es 10.

    Retorna:
    - Número óptimo de clusters.
    """

    # Calcular la linkage matrix
    mergings = linkage(features, method=method)

    # Obtener las alturas de los últimos 'last_n' clusters
    last = mergings[-last_n:, 2]
    last_rev = last[::-1]

    # Calcular la aceleración (segunda derivada)
    acceleration = np.diff(last, 2)  # Segunda derivada de las alturas
    acceleration_rev = acceleration[::-1]

    # Encontrar el número óptimo de clusters
    optimal_k = acceleration_rev.argmax() + 2  # +2 porque se pierde una posición en cada derivada

    return optimal_k


## carga de datos 

In [2]:
file_path=data_dir("interim","estadisticas_edificios.csv")

In [3]:
data = pd.read_csv(file_path)
data.head()

,type_building,var1_mean,var1_std_dev,var2_mean,var2_std_dev
0,RefBldgFullServiceRestaurantNew2004,0.000000,6.226848,19.4245,7.265027
1,RefBldgFullServiceRestaurantNew2004,0.001580,6.596764,19.4245,7.265027
2,RefBldgFullServiceRestaurantNew2004,0.002568,7.146033,19.4245,7.265027
3,RefBldgFullServiceRestaurantNew2004,0.000000,4.688730,19.4245,7.265027
4,RefBldgFullServiceRestaurantNew2004,0.000000,4.797245,19.4245,7.265027


## CLUSTERING 

In [6]:
data_clustered=data.copy()

In [13]:
params=optimal_dbscan_params(X, eps_range=(0.05, 0.2, 0.05), min_samples_range=(3, 12))
params

(np.float64(0.2), np.int64(3))

In [14]:
# Aplicar DBSCAN
dbscan = DBSCAN(eps=params[0], min_samples=params[1])
clusters_dbscan = dbscan.fit_predict(X)

# Agregar los clusters al DataFrame para análisis posterior
data_clustered['Cluster_DBSCAN'] = clusters_dbscan
# Mostrar los primeros registros con los clusters asignados por DBSCAN
data_clustered.head()

,type_building,var1_mean,var1_std_dev,var2_mean,var2_std_dev,Cluster_KMeans,Cluster_DBSCAN
0,RefBldgFullServiceRestaurantNew2004,0.000000,6.226848,19.4245,7.265027,0,0
1,RefBldgFullServiceRestaurantNew2004,0.001580,6.596764,19.4245,7.265027,0,0
2,RefBldgFullServiceRestaurantNew2004,0.002568,7.146033,19.4245,7.265027,0,0
3,RefBldgFullServiceRestaurantNew2004,0.000000,4.688730,19.4245,7.265027,0,0
4,RefBldgFullServiceRestaurantNew2004,0.000000,4.797245,19.4245,7.265027,0,0


**Silueta**
El coeficiente de silueta mide cuán similares son los objetos dentro de un mismo cluster comparados con objetos de otros clusters. Va de -1 a 1, donde valores cercanos a 1 indican buenos clusters, cercanos a 0 indican clusters solapados y valores negativos indican asignaciones incorrectas.

In [15]:
# Calcular el silhouette score para evaluar la calidad de los clusters, excluyendo los puntos ruidosos
if len(set(clusters_dbscan)) > 1:
    silhouette_dbscan = silhouette_score(X, clusters_dbscan)
else:
    silhouette_dbscan = -1  # Silhouette score no es aplicable si hay un solo cluster
silhouette_dbscan

np.float64(0.6378114591428399)

**Índice de Davies-Bouldin**
Mide la compactación de los clusters y la separación entre ellos. Un valor más bajo indica una mejor formación de clusters.

In [16]:
# Calcular el índice de Davies-Bouldin
davies_bouldin_dbscan= davies_bouldin_score(X, clusters_dbscan)
davies_bouldin_dbscan

np.float64(0.8368992004306053)

In [29]:
def clustering_analysis(data, max_k=10, eps_range=(0.05, 0.2, 0.05), min_samples_range=(3, 12)):
    # Estandarización de los datos
    numeric_columns = data.select_dtypes(include=['float64']).columns
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data[numeric_columns])
    X = pd.DataFrame(data_scaled, columns=numeric_columns)
    
    # DataFrame para almacenar métricas de cada modelo
    metrics = pd.DataFrame(columns=["Model", "Silhouette Score", "Davies-Bouldin Index"])

    # K-Means Clustering
    optimal_k_kmeans = optimal_k_selection(X, max_k=max_k)
    kmeans = KMeans(n_clusters=optimal_k_kmeans, random_state=42)
    clusters_kmeans = kmeans.fit_predict(X)
    silhouette_kmeans = silhouette_score(X, clusters_kmeans)
    davies_bouldin_kmeans = davies_bouldin_score(X, clusters_kmeans)
    
    # Añadir métricas de K-Means al DataFrame
    metrics = pd.concat([metrics, pd.DataFrame({
        "Model": ["K-Means"],
        "Silhouette Score": [silhouette_kmeans],
        "Davies-Bouldin Index": [davies_bouldin_kmeans]
    })], ignore_index=True)

    # DBSCAN Clustering
    eps_min_samples = optimal_dbscan_params(X, eps_range=eps_range, min_samples_range=min_samples_range)
    dbscan = DBSCAN(eps=eps_min_samples[0], min_samples=eps_min_samples[1])
    clusters_dbscan = dbscan.fit_predict(X)

    # Calculamos las métricas solo si hay más de un cluster
    if len(set(clusters_dbscan)) > 1:
        silhouette_dbscan = silhouette_score(X, clusters_dbscan)
    else:
        silhouette_dbscan = -1  # Silhouette score no es aplicable si hay un solo cluster
    davies_bouldin_dbscan = davies_bouldin_score(X, clusters_dbscan)

    # Añadir métricas de DBSCAN al DataFrame
    metrics = pd.concat([metrics, pd.DataFrame({
        "Model": ["DBSCAN"],
        "Silhouette Score": [silhouette_dbscan],
        "Davies-Bouldin Index": [davies_bouldin_dbscan]
    })], ignore_index=True)

    # Clustering Jerárquico
    optimal_k_hierarchical = optimal_clusters_hierarchical(X, method='ward', last_n=10)
    hierarchical = AgglomerativeClustering(n_clusters=optimal_k_hierarchical)
    clusters_hierarchical = hierarchical.fit_predict(X)
    silhouette_hierarchical = silhouette_score(X, clusters_hierarchical)
    davies_bouldin_hierarchical = davies_bouldin_score(X, clusters_hierarchical)

    # Añadir métricas de clustering jerárquico al DataFrame
    metrics = pd.concat([metrics, pd.DataFrame({
        "Model": ["Hierarchical"],
        "Silhouette Score": [silhouette_hierarchical],
        "Davies-Bouldin Index": [davies_bouldin_hierarchical]
    })], ignore_index=True)

    # Normalización de las métricas para la selección del mejor modelo
    metrics["Silhouette Score Norm"] = (metrics["Silhouette Score"] - metrics["Silhouette Score"].min()) / (metrics["Silhouette Score"].max() - metrics["Silhouette Score"].min())
    metrics["Davies-Bouldin Index Norm"] = (metrics["Davies-Bouldin Index"].max() - metrics["Davies-Bouldin Index"]) / (metrics["Davies-Bouldin Index"].max() - metrics["Davies-Bouldin Index"].min())
    
    # Cálculo de la puntuación combinada (promedio de ambas métricas normalizadas)
    metrics["Combined Score"] = metrics[["Silhouette Score Norm", "Davies-Bouldin Index Norm"]].mean(axis=1)

    # Selección del mejor modelo según la puntuación combinada
    best_model = metrics.loc[metrics["Combined Score"].idxmax()]

    return metrics, best_model


In [32]:
metrics,best_models=clustering_analysis(data, max_k=10, eps_range=(0.05, 0.2, 0.05), min_samples_range=(3, 12))

C:\Users\ASUS TUF\.conda\envs\energy_consumption_architecture\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\ASUS TUF\.conda\envs\energy_consumption_architecture\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\ASUS TUF\.conda\envs\energy_consumption_architecture\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\ASUS TUF\.conda\

In [33]:
metrics

,Model,Silhouette Score,Davies-Bouldin Index,Silhouette Score Norm,Davies-Bouldin Index Norm,Combined Score
0,K-Means,0.832731,0.171948,1.000000,1.000000,1.000000
1,DBSCAN,0.637811,0.836899,0.000000,0.000000,0.000000
2,Hierarchical,0.762659,0.699451,0.640507,0.206704,0.423605


In [36]:
best_models

Model                         K-Means
Silhouette Score             0.832731
Davies-Bouldin Index         0.171948
Silhouette Score Norm             1.0
Davies-Bouldin Index Norm         1.0
Combined Score                    1.0
Name: 0, dtype: object